# Pomnażanie obrazów i przenoszenie nauki

In [1]:
import os

# Ustawienie PyTorch jako zaplecza dla biblioteki Keras
os.environ["KERAS_BACKEND"] = "torch"
import torch
import keras
from keras import layers, datasets

import numpy as np
import PIL.Image as Image
import matplotlib.pylab as plt

# Sprawdzenie dostępności GPU
#print(tf.config.list_physical_devices('GPU'))
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    if torch.xpu.is_available():
        for i in range(torch.xpu.device_count()):
            print(f"XPU {i}: {torch.xpu.get_device_name(i)}")
    else:
        print("Brak dostępnych GPU lub XPU.")
        print("Używam CPU.")

GPU 0: Tesla T4


In [4]:
IMAGE_SIZE = 96 #299
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
num_classes = 10

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [5]:
def build_model(classifier):

    classifier.trainable = True  # Unfreeze the base model for fine-tuning
    # Create new model on top
    inputs = keras.Input(shape=(28, 28, 1))
    x = keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE)(inputs)  # Resize input to match the base model
    x = keras.layers.Concatenate()([x,x,x])
    # Simple data augmentation
    x = keras.layers.RandomFlip("horizontal")(x)

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = classifier(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(num_classes)(x)
    model = keras.Model(inputs, outputs)

    model.summary(show_trainable=True)

    return model

finetune_net = build_model(keras.applications.ResNet50V2(
                                                    weights="imagenet",
                                                    include_top=False,
                                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))

scratch_net = build_model(keras.applications.ResNet50V2(
                                                    weights=None,
                                                    include_top=False,
                                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)      ┃ Output Shape    ┃   Param # ┃ Connected to   ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1     │ (None, 28, 28,  │         0 │ -              │   -   │
│ (InputLayer)      │ 1)              │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ resizing          │ (None, 96, 96,  │         0 │ input_layer_1… │   -   │
│ (Resizing)        │ 1)              │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ concatenate       │ (None, 96, 96,  │         0 │ resizing[0][0… │   -   │
│ (Concatenate)     │ 3)              │           │ resizing[0][0… │       │
│                   │                 │           │ resizing[0][0] │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ random_flip       │ (None, 96, 96,  │         0 │ concatenate[0… │   -   │
│ (RandomFlip)      │ 3)              │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ resnet50v2        │ (None, 3, 3,    │ 23,564,8… │ random_flip[0… │   Y   │
│ (Functional)      │ 2048)           │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ global_average_p… │ (None, 2048)    │         0 │ resnet50v2[0]… │   -   │
│ (GlobalAveragePo… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ dropout (Dropout) │ (None, 2048)    │         0 │ global_averag… │   -   │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ dense (Dense)     │ (None, 10)      │    20,490 │ dropout[0][0]  │   Y   │
└───────────────────┴─────────────────┴───────────┴────────────────┴───────┘

 Total params: 23,585,290 (89.97 MB)

 Trainable params: 23,539,850 (89.80 MB)

 Non-trainable params: 45,440 (177.50 KB)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)      ┃ Output Shape    ┃   Param # ┃ Connected to   ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_3     │ (None, 28, 28,  │         0 │ -              │   -   │
│ (InputLayer)      │ 1)              │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ resizing_1        │ (None, 96, 96,  │         0 │ input_layer_3… │   -   │
│ (Resizing)        │ 1)              │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ concatenate_1     │ (None, 96, 96,  │         0 │ resizing_1[0]… │   -   │
│ (Concatenate)     │ 3)              │           │ resizing_1[0]… │       │
│                   │                 │           │ resizing_1[0]… │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ random_flip_1     │ (None, 96, 96,  │         0 │ concatenate_1… │   -   │
│ (RandomFlip)      │ 3)              │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ resnet50v2        │ (None, 3, 3,    │ 23,564,8… │ random_flip_1… │   Y   │
│ (Functional)      │ 2048)           │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ global_average_p… │ (None, 2048)    │         0 │ resnet50v2[0]… │   -   │
│ (GlobalAveragePo… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ dropout_1         │ (None, 2048)    │         0 │ global_averag… │   -   │
│ (Dropout)         │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ dense_1 (Dense)   │ (None, 10)      │    20,490 │ dropout_1[0][… │   Y   │
└───────────────────┴─────────────────┴───────────┴────────────────┴───────┘

 Total params: 23,585,290 (89.97 MB)

 Trainable params: 23,539,850 (89.80 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [ ]:
# Compile the model for multiclass classification
scratch_net.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# Train the model
history = scratch_net.fit(
    train_images, train_labels,
    epochs=10,
    batch_size=128,
    validation_data=(test_images, test_labels),
)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = scratch_net.evaluate(test_images,  test_labels, verbose=2)

## Zadania

1. Wytrenuj model bez pomnażania obrazów. Porównaj celność treningową jak też i testową z oraz bez pomnażania obrazów. Czy ten eksperyment porównawczy popiera argument, że pomnażanie obrazów może ograniczać nadmierne dopasowanie? Dlaczego?
2. Wymieszaj różne metody poszerzania zbiorów danych na zbiorze CIFAR-10. Czy poprawiaja one celność?
3. Zwiększ współczynnik uczenia w `finetune_net`. Jak zmieni się celność predykcji modelu?
4. Kontynuuj dostosowywanie hiperparametrów obu sieci w eksperymencie porównawczym. Czy nadal ich trafność predykcji różni się?
5. Ustaw parametry wszystkich warstw z wyjątkiem wyjściowej w siei `finetune_net` na te ze źródłowego modelu i nie aktualizuj ich podczas trningu. Jak zmieni się celność modelu?